# Model

In [1]:
!pip install ultralytics

In [1]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
model = YOLO("yolov8n.pt")
# image_path1 = ".\MarsRoverDataset\Test\Obstacles\Obstacle_51.jpg" # Obstakels
# image_path2 = ".\MarsRoverDataset\Test\FreePath\FreePath_54.jpg" # Vrije paden

# results = model(image_path1)

# # results[0].show()

# obstacles = results[0].boxes
# if len(obstacles) > 0:
#     print("Obstakel")
# else:
#     print("Vrij pad")

# Retrieving Camera Footage

In [3]:
import cv2

In [4]:
# for i in range(5):
#     cap = cv2.VideoCapture(i)
#     if cap.isOpened():
#         print(f"Camera is gevonden op index {i}")
#     else:
#         print(f"Géén camera op index {i}")
#     cap.release()
#Kijken op welke indexes camera's zijn gevonden

In [5]:
from ultralytics import YOLO
import time

In [6]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.


In [7]:
import serial
import time

In [8]:
arduino = serial.Serial("COM3", 9600)
time.sleep(2)

In [9]:
# arduino.write(b"\n")

In [10]:
camera_index = 2
# Model eerder gecreëerd
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print("Kan de camera niet openen")
    exit()

def is_obstacle_detected(results, class_filter={"person", "car", "chair"}, size_thresh=0.05):
    img_w, img_h = results[0].orig_shape[1], results[0].orig_shape[0]
    for box in results[0].boxes:
        cls = int(box.cls[0])
        name = model.names[cls]
        if name not in class_filter:
            continue
        x1, y1, x2, y2 = box.xyxy[0]
        box_area = (x2 - x1) * (y2 - y1)
        img_area = img_w * img_h
        if box_area / img_area > size_thresh:
            return True, name
    return False, None

# 🌀 Live loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Geen frame ontvangen")
        break

    results = model(frame, verbose=False) # Verbose op False zorgt ervoor dat er geen output ontstaat van 1000 regels

    # Analyseer
    obstacle, obj_name = is_obstacle_detected(results)
    if obstacle:
        status = f"OBSTAKEL GEDTECTEERD ({obj_name})"
        arduino.write(b"Obstacle\n")
    else:
        status = "VRIJ PAD"
        arduino.write(b"FreePath\n")

    # Teken resultaten + status
    annotated = results[0].plot()
    cv2.putText(annotated, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0) if not obstacle else (0, 0, 255), 2)

    # Toon beeld
    cv2.imshow("FPV Live Feed", annotated)

    # Stop met Q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 🧹 Opruimen
cap.release()
cv2.destroyAllWindows()

SerialException: WriteFile failed (PermissionError(13, 'Het apparaat herkent de opdracht niet.', None, 22))

# Arduino communicatie

In [ ]:
# if obstacle:
#     print("Obstacle")
#     arduino.write(b"Obstacle\n")
# else:
#     print("FreePath")
#     arduino.write(b"FreePath\n")